# Data Exploration


In this notebook, we will introduce several plotting methods to help you analyze the stake measurement dataset you created during data preparation. The `MassBalanceMachine` library currently provides two types of plotting methods:

First, it allows you to plot a point surface mass balance timeseries for all stakes across all measured years. This includes the region-wise mean and standard deviation of the point surface mass balance. Second, it enables you to plot a region-wide cumulative annual surface mass balance for the available stakes in the dataset.

We plan to add more data visualization options in future releases of `MassBalanceMachine`. If you have suggestions for new visualizations that would benefit other users or are specific to a certain region, please feel free to create new GitHub issues or submit pull requests.

**Note:** It is assumed that the column names in the dataset resemble those in the WGMS files. If this isn't the case, please refer to either the [data preprocessing notebook](https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_preprocessing.ipynb) or the [data preparation notebook](https://github.com/ODINN-SciML/MassBalanceMachine/blob/main/notebooks/data_processing_wgms.ipynb) for guidance.

In [ ]:
import pandas as pd
import massbalancemachine as mbm

In [ ]:
data = pd.read_csv('./example_data/iceland/files/iceland_monthly_dataset.csv')
display(data)

## 1. Plot the Point Surface Mass Balance for all Available Stakes

In this section, we will generate a plot for the point surface mass balance timeseries for all available stakes across all measured years. The plot will also include the region-wise mean and standard deviation of the point surface mass balance, providing a comprehensive view of the data's temporal and regional trends.

In [ ]:
mbm.data_processing.utils.plot_stake_timeseries(data, ['svg', './example_data/iceland/plots/'])

## 2. Plot the Cumulative Region-Wide Surface Mass Balance

In this section, we will create a plot to visualize the cumulative annual surface mass balance for a single region. This plot will aggregate the data from all available stakes within the region, offering insights into the overall mass balance trends over time.

In [ ]:
mbm.data_processing.utils.plot_cumulative_smb(data, ['svg', './example_data/iceland/plots/'])